In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from time import sleep
from time import time
from random import randint
from IPython.core.display import clear_output
import pandas as pd
import numpy as np
import psycopg2
import requests
import os
import re
import ast

In [2]:
headers = {'User-Agent': 'Mozilla/5.0',
           'From': 'itchyandscratchy@gmail.com'
          }

data_path = '/Users/amanda/Documents/Projects/insight/data/'

In [3]:
url = 'https://noc.esdc.gc.ca/Structure/Hierarchy/1a06c040a8bd4022ac498a80eb089bc1?objectid=%2Fd0IGA6qD8JPRfoj5UCjpg%3D%3D'

joblinks = []
# Main summary page
with requests.get(url, timeout=5, headers=headers) as u:     

    soup = BeautifulSoup(u.content)
    for link in soup.find_all('a'):
        joblinks.append(link.get('href'))
            
linkends = [s for s in joblinks if 'NocProfile' in s]            

In [4]:
print(linkends[0])

/Structure/NocProfile/d86ebb9a0ef646f6ad0ef3d58f5ffcbd?objectid=NmDF%2Bp5XjPpmMEzWh5irOIJPO4DWI3AoLXAGtO9E%2F%2Fr2xVJHKlh2Vvm4wy6CUIjy


In [13]:
base_url = 'https://noc.esdc.gc.ca'

overview_df = pd.DataFrame({'noc_link':[],
                            'noc_code':[],
                            'job_group':[],
                            'description':[],
                            'alt_titles':[]})

duties_df = pd.DataFrame({'noc_code':[],
                          'duties':[],
                          'requirements':[]})

for link in linkends:
    
    try:

        # Main summary page
        url = base_url + link
        with requests.get(url, timeout=8, headers=headers) as u:     
            soup = BeautifulSoup(u.content) 
            #print(soup.prettify())

            # Main website content
            a = soup.find('main', attrs={'class':'container'})

            # Title and overview section
            title = a.find('h2',attrs={'style':'margin-top:25px;margin-bottom:0;'})
            intro = a.find('p', attrs={'class':'mrgn-tp-md'})

            if title is not None and intro is not None:
                noc_summary = title.text.split('–') #Note this is an en-dash, not a hyphen
                noc_code = noc_summary[0]
                job_group = noc_summary[-1]
                print(job_group)
                job_intro = intro.text.strip()
            else:
                continue

            title_sec = a.find('div',attrs={'id':'IndexTitles'})
            title_list = title_sec.findAll('li')
            alt_titles = [s.text.strip() for s in title_list]

            overview_df = overview_df.append({'noc_link':link,
                                              'noc_code':noc_code,
                                              'job_group':job_group,
                                              'description':job_intro,
                                              'alt_titles':alt_titles}, ignore_index=True)

            # Duties and requirements
            job_duties = []
            duty_sec = soup.find('h4', text = 'Main duties').findNext('div', attrs={'class':'panel-body'})
            duty_list = duty_sec.findAll('li')
            if duty_list:
                job_duties = [s.text.strip() for s in duty_list]
            else:
                job_duties = 'NaN'

            job_req = []
            req_head = soup.find('h4', text = 'Employment requirements').findNext('ul')
            req_list = req_head.findAll('li')
            if req_list:
                job_reqs = [s.text.strip() for s in req_list]
            else:
                job_reqs = 'NaN'

            duties_df = duties_df.append({'noc_code':noc_code,
                                          'duties':job_duties,
                                          'requirements':job_reqs}, ignore_index=True)            
            
    except:
        continue
    
 

  Legislators
  Senior government managers and officials
  Senior managers - financial, communications and other business services
  Senior managers - health, education, social and community services and membership organizations
  Senior managers - trade, broadcasting and other services, n.e.c.
  Senior managers - construction, transportation, production and utilities
  Financial managers
  Human resources managers
  Purchasing managers
  Other administrative services managers
  Insurance, real estate and financial brokerage managers
  Banking, credit and other investment managers
  Advertising, marketing and public relations managers
  Other business services managers
  Telecommunication carriers managers
  Postal and courier services managers
  Engineering managers
  Architecture and science managers
  Computer and information systems managers
  Managers in health care
  Government managers - health and social policy development and program administration
  Government managers - econ

  Other assisting occupations in support of health services
  University professors and lecturers
  Post-secondary teaching and research assistants
  College and other vocational instructors
  Secondary school teachers
  Elementary school and kindergarten teachers
  Educational counsellors
  Judges
  Lawyers and Quebec notaries
  Psychologists
  Social workers
  Family, marriage and other related counsellors
  Professional occupations in religion
  Probation and parole officers and related occupations
  Employment counsellors
  Natural and applied science policy researchers, consultants and program officers
  Economists and economic policy researchers and analysts
  Business development officers and marketing researchers and consultants
  Social policy researchers, consultants and program officers
  Health policy researchers, consultants and program officers
  Education policy researchers, consultants and program officers
  Recreation, sports and fitness policy researchers, consultants

  Supervisors, logging and forestry
  Supervisors, mining and quarrying
  Contractors and supervisors, oil and gas drilling and services
  Underground production and development miners
  Oil and gas well drillers, servicers, testers and related workers
  Logging machinery operators
  Agricultural service contractors, farm supervisors and specialized livestock workers
  Contractors and supervisors, landscaping, grounds maintenance and horticulture services
  Fishing masters and officers
  Fishermen/women
  Underground mine service and support workers
  Oil and gas well drilling and related workers and services operators
  Chain saw and skidder operators
  Silviculture and forestry workers
  General farm workers
  Nursery and greenhouse workers
  Fishing vessel deckhands
  Trappers and hunters
  Harvesting labourers
  Landscaping and grounds maintenance labourers
  Aquaculture and marine harvest labourers
  Mine labourers
  Oil and gas drilling, servicing and related labourers
  Logging 

In [14]:
# Save cleaned dataframes to file 

overview_df.to_csv(os.path.join(data_path,'processed','noc-overview.csv'), index=False)


In [52]:
# Try urls to find out if they exist
# Note this is a long process and only generally needs to be run once

url_list = []

for i in range(1,30000):  
    try:
        url = 'https://www.jobbank.gc.ca/marketreport/summary-occupation/{}/ca'.format(i)  
        request = requests.get(url, headers=headers, timeout=5, allow_redirects=False)
        if request.status_code == 200:
            url_list.append(url)
            print(url)
        else:
            print('SKIP')
    except:
        continue

https://www.jobbank.gc.ca/marketreport/summary-occupation/1/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/2/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/3/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/4/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/5/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/6/ca
https://www.jobbank.gc.ca/marketreport/summary-occupation/7/ca
SKIP
https://www.jobbank.gc.ca/marketreport/summary-occupation/9/ca


In [51]:
clean_job_urls = [x.replace('https://www.jobbank.gc.ca/marketreport/summary-occupation','') for x in url_list]

In [50]:
# Save url_list to file so I can load it later

file_path = (os.path.join(data_path,'processed','job_url_list.txt'))

with open(file_path, 'w') as filehandle:
    for listitem in clean_job_urls:
        filehandle.write('%s\n' % listitem)